In [1]:
import pandas as pd
import numpy as np
#import standardscaler and label encoder for normalization of data and encoding of class
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

training_activity = train['Activity']
testing_activity = test['Activity']
train = train.iloc[:,:260]
test = test.iloc[:,:260]
#filter data
train = train.drop(train.filter(regex='meanFreq|Jerk|Mag'),axis=1)
test = test.drop(test.filter(regex='meanFreq|Jerk|Mag'),axis=1)

train = train.filter(regex='mean|std')
test = test.filter(regex='mean|std')

print(train)
##print(test)

      tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.288585          -0.020294          -0.132905   
1              0.278419          -0.016411          -0.123520   
2              0.279653          -0.019467          -0.113462   
3              0.279174          -0.026201          -0.123283   
4              0.276629          -0.016570          -0.115362   
...                 ...                ...                ...   
7347           0.299665          -0.057193          -0.181233   
7348           0.273853          -0.007749          -0.147468   
7349           0.273387          -0.017011          -0.045022   
7350           0.289654          -0.018843          -0.158281   
7351           0.351503          -0.012423          -0.203867   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  \
0            -0.995279         -0.983111         -0.913526   
1            -0.998245         -0.975300         -0.960322   
2            -0.995380         -0

In [3]:
#now scale the data between 1 to -1
scaler = StandardScaler().fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

#now encode the activity
le = LabelEncoder().fit(['STANDING','SITTING','LAYING','WALKING','WALKING_UPSTAIRS','WALKING_DOWNSTAIRS'])
training_act_enc = le.transform(training_activity)
testing_act_enc = le.transform(testing_activity)

print(training_act_enc)
print(testing_act_enc)

[2 2 2 ... 5 5 5]
[2 2 2 ... 5 5 5]


In [4]:
#train the model
svm = SVC(kernel='linear')
dt = dt()
xt = ExtraTreesClassifier()
rf = rf()

kf = KFold(n_splits=10)

def evaluate_model(model):
    score = cross_val_score(model,train,training_act_enc,cv=kf)
    average = score.mean()
    return "Performance : {} \n Accuracy:{}".format(score, average)

print("ExtraTree Classifier : {}".format(evaluate_model(xt)))
print("Random Forest Classifier : {}".format(evaluate_model(rf)))

ExtraTree Classifier : Performance : [0.91032609 0.9076087  0.92517007 0.83537415 0.87346939 0.79863946
 0.8462585  0.8585034  0.90612245 0.86258503] 
 Accuracy:0.8724057231588287
Random Forest Classifier : Performance : [0.88315217 0.89266304 0.9292517  0.83945578 0.84081633 0.78639456
 0.84353741 0.84353741 0.90884354 0.84489796] 
 Accuracy:0.8612549911268855


In [5]:
#Extra Tree Classifier has greater accuracy than all of other. so we will further improve its performance by
#tuning the algorithm
from sklearn.model_selection import RandomizedSearchCV as randomcv

parameters = {'n_estimators':np.arange(20,200,20), 'min_samples_split':np.arange(2,10,2)}
model = ExtraTreesClassifier()
search = randomcv(model,param_distributions=parameters,n_iter=30,n_jobs=4)

search.fit(train,training_act_enc)
print(search.best_score_)
print(search.best_estimator_)

0.8812548268797663
ExtraTreesClassifier(min_samples_split=8, n_estimators=180)


In [30]:
model = search.best_estimator_
model.fit(train,training_act_enc)
prediction = model.predict(test)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(classification_report(prediction, testing_act_enc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       537
           1       0.87      0.90      0.89       471
           2       0.92      0.89      0.90       550
           3       0.82      0.78      0.80       523
           4       0.73      0.85      0.78       358
           5       0.77      0.71      0.74       508

    accuracy                           0.86      2947
   macro avg       0.85      0.86      0.85      2947
weighted avg       0.86      0.86      0.86      2947



In [47]:
import json
sensordata = []
with open('sensor.txt') as frt:
    for line in frt:
        sensordata.append(json.loads(line))
        
Gyros = []
asscelo = []
glist = []
alist = []
for x in sensordata:
    if x['Sensor_Type'] == 'GyroScope':
        Gyros = []
        Gyros.append(x['val1'])
        Gyros.append(x['val2'])
        Gyros.append(x['val3'])
        if(len(glist) <= 128):
            glist.append(Gyros)
    else:
        asscelo= []
        asscelo.append(x['val1'])
        asscelo.append(x['val2'])
        asscelo.append(x['val3'])
        if(len(glist) <= 128):
            alist.append(asscelo)
        
linearaccel = []

for accel in alist:
    g0 = 9.81
    g1 = 9.81
    g2 = 9.81
    alpha = 0.8
    g0 = alpha * g0 + (1 - alpha) * accel[0];
    g1 = alpha * g1 + (1 - alpha) * accel[1];
    g2 = alpha * g2 + (1 - alpha) * accel[2];
    linear = []
    linear.append(accel[0]-g0)
    linear.append(accel[1]-g1)
    linear.append(accel[2]-g2)
    linearaccel.append(linear)
  

scaler = StandardScaler().fit(alist)
alist = scaler.transform(alist)

scaler1 = StandardScaler().fit(linearaccel)
linearaccel = scaler1.transform(linearaccel)
scaler2 = StandardScaler().fit(glist)
glist = scaler2.transform(glist)
accel1 = alist[:,:1]
accel2 = alist[:,1:2]
accel3 = alist[:,2:]

accel1mean = accel1.mean()
accel2mean = accel2.mean()
accel3mean = accel3.mean()
accel1std = accel1.std()
accel2std = accel2.std()
accel3std = accel3.std()
print(accel1mean,accel1std)
print(accel2mean,accel2std)
print(accel3mean,accel3std)

laccel1 = linearaccel[:,:1]
laccel2 = linearaccel[:,1:2]
laccel3 = linearaccel[:,2:]

laccel1mean = laccel1.mean()
laccel2mean = laccel2.mean()
laccel3mean = laccel3.mean()
laccel1std = laccel1.std()
laccel2std = laccel2.std()
laccel3std = laccel3.std()
print(laccel1mean,laccel1std)
print(laccel2mean,laccel2std)
print(laccel3mean,laccel3std)
gyro1= glist[:,:1]
gyro2 = glist[:,1:2]
gyro3 = glist[:,2:]

gyro1mean = gyro1.mean()
gyro2mean = gyro2.mean()
gyro3mean = gyro3.mean()
gyro1std = gyro1.std()
gyro2std = gyro2.std()
gyro3std = gyro3.std()
print(gyro1mean,gyro1std)
print(gyro2mean,gyro2std)
print(gyro3mean,gyro3std)
    
tst = [[laccel1mean,laccel2mean,laccel3mean,laccel1std,laccel2std,laccel3std,accel1mean,accel2mean,accel3mean,accel1std,accel2std,accel3std,gyro1mean,gyro2mean,gyro3mean,gyro1mean,gyro2mean,gyro3mean]]

2.1997218861239767e-15 1.0
1.7763568394002505e-17 1.0000000000000002
-1.6511236822225328e-14 1.0000000000000004
-2.3320604706592955e-13 1.0000000000000002
4.264440652453535e-14 1.0
-2.102022259956963e-16 1.0000000000000002
1.200241107702872e-17 0.9999999999999998
-5.401084984662924e-17 1.0000000000000013
-2.100421938480026e-17 1.0


In [48]:


prediction = model.predict(tst)

In [49]:
print(prediction)

[5]


In [50]:
print(tst)

[[-2.3320604706592955e-13, 4.264440652453535e-14, -2.102022259956963e-16, 1.0000000000000002, 1.0, 1.0000000000000002, 2.1997218861239767e-15, 1.7763568394002505e-17, -1.6511236822225328e-14, 1.0, 1.0000000000000002, 1.0000000000000004, 1.200241107702872e-17, -5.401084984662924e-17, -2.100421938480026e-17, 1.200241107702872e-17, -5.401084984662924e-17, -2.100421938480026e-17]]
